# Scenario 2: A single data scientist participating in an ML competition

MLflow setup:

- Tracking server: yes, local server
- Backend store: sqlite database
- Artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1703147935108, experiment_id='1', last_update_time=1703147935108, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1703147681328, experiment_id='0', last_update_time=1703147681328, lifecycle_stage='active', name='Default', tags={}>]

## Creating an experiment and logging a new run

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: '/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/b2e824f86b3a4bc0b5e15734bdf8131a/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1703147935108, experiment_id='1', last_update_time=1703147935108, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1703147681328, experiment_id='0', last_update_time=1703147681328, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [22]:
client.search_registered_models()


[]

In [23]:
runs = client.search_runs(
    experiment_ids="1",
    max_results=3,
)

In [24]:
run = runs[0]
run_id = run.info.run_id
print(run_id)

b2e824f86b3a4bc0b5e15734bdf8131a


In [14]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

AttributeError: 'MlflowClient' object has no attribute 'list_run_infos'